In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-06-26 16:11:17--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-06-26 16:11:17--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccdc659d85106c2d4822de1eb49.dl.dropboxusercontent.com/cd/0/inline/Bn_bh14QufzzqJQb9w9spekoOQSdjzNxO3meIbjLv4KyqK7iJtg0Hj7oYFbdbBAA3Sw4DTWtwLg1_RQy9sn5abMTNMN1CXaKqv4kbH4opXwNK5RDLKjAwdhWHOEgrY5z9Eux3CMBIy4zlwLvSCR6-TzYwtn373C2c5MKWZ73vGHAuw/file# [following]
--2022-06-26 16:11:18--  https://uccdc659d85106c2d4822de1eb49.dl.dropboxusercontent.com/cd/0/inline/Bn_bh14QufzzqJQb9w9spekoOQ

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
# Sentiment Analysis

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Bidirectional

In [5]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />', ' '), label)
  )

In [6]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Rock Star: INXS was the best music TV series I have ever watched! It had some of the greatest rock n' roll songs ever written, performed by 15 very talented singers/performers. It also had (in my opinion) the most heart-felt, feel-good, surprise endings in all of reality TV. It actually made me shed tears of happiness for the winner!!! Over the 13 weeks of this televised competition, the viewing audience got to know and became familiar with all of the contestants. After 30-some episodes the remaining contestants seemed more like friends than just some more strangers competing against each other on a reality TV show. And the fact that INXS was, and still is, one of the greatest rock n' roll bands EVER just added to the emotional tension created by this wonderful reality series. If you don't have the series recorded, ROCK STAR: INXS the DVD is a great alternative."
1


In [7]:
model = Sequential()

In [8]:
model.add(Input(shape=(1,), dtype="string"))

In [9]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len
)

In [10]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [11]:
model.add(Embedding(max_tokens + 1, 128))

model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [12]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 24s 20ms/step - loss: 0.5092 - accuracy: 0.7440
Epoch 2/10
782/782 [==============================] - 15s 19ms/step - loss: 0.4271 - accuracy: 0.8048
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.3955 - accuracy: 0.8200
Epoch 4/10
782/782 [==============================] - 15s 19ms/step - loss: 0.3643 - accuracy: 0.8393
Epoch 5/10
782/782 [==============================] - 15s 20ms/step - loss: 0.3373 - accuracy: 0.8524
Epoch 6/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3063 - accuracy: 0.8669
Epoch 7/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2776 - accuracy: 0.8812
Epoch 8/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2526 - accuracy: 0.8932
Epoch 9/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2255 - accuracy: 0.9062
Epoch 10/10
782/782 [==============================] - 16s 20ms/step - loss: 0.207

In [14]:
model.evaluate(test_data)

782/782 [==============================] - 11s 13ms/step - loss: 0.7058 - accuracy: 0.7686


[0.7057775855064392, 0.768559992313385]

In [15]:
text = "I loved the movie !"

In [16]:
model.predict([text])

array([[0.9247301]], dtype=float32)